In [20]:
import json
import subprocess
import httpx
from typing import Any

def get_sf_conn(username: str = "username") -> dict[str, str]:
    """Run Salesforce CLI to get connection info (accessToken, instanceUrl, apiVersion)."""
    result = subprocess.run(
        ["sf", "force", "org", "display", "-u", username, "--json"],
        capture_output=True,
        text=True
    )
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    if result.returncode != 0:
        raise RuntimeError(f"sf CLI failed: {result.stderr}")
    data = json.loads(result.stdout)
    return {
        "access_token": data["result"]["accessToken"],
        "instance_url": data["result"]["instanceUrl"],
        "api_version": data["result"]["apiVersion"]
    }

async def run_flow(
    flow_api_name: str,
    input_variables: dict[str, Any],
    sf_conn: dict[str, str]
) -> dict[str, Any]:
    """Invoke a Salesforce Flow via REST API."""
    url = f"{sf_conn['instance_url']}/services/data/v{sf_conn['api_version']}/actions/custom/flow/{flow_api_name}"
    headers = {
        "Authorization": f"Bearer {sf_conn['access_token']}",
        "Content-Type": "application/json"
    }
    payload = {"inputs": [input_variables]}
    async with httpx.AsyncClient() as client:
        resp = await client.post(url, headers=headers, json=payload, timeout=30.0)
        resp.raise_for_status()
        return resp.json()


In [21]:
auth = get_sf_conn('mohan.chinnappan.n.ea10@gmail.com')
print(auth)

STDOUT: {
  "status": 0,
  "result": {
    "id": "00DHs000000QASYMA4",
    "apiVersion": "63.0",
    "accessToken": "00DHs000000QASY!AQwAQIeGppSLVouVSdgZDTOQuGPUEJCajjqKjFnxVScxIBU2rurdKV6bP_AheN3ba3z_SI95icFFu.MS.K36XAxJS2reVTYH",
    "instanceUrl": "https://dhs000000qasyma4-dev-ed.develop.my.salesforce.com",
    "username": "mohan.chinnappan.n.ea10@gmail.com",
    "clientId": "PlatformCLI",
    "connectedStatus": "Connected"
  },
  "warnings": [
    "The \"-u\" flag has been deprecated. Use \"--target-org | -o\" instead.",
    "This command will expose sensitive information that allows for subsequent activity using your current authenticated session.\nSharing this information is equivalent to logging someone in under the current credential, resulting in unintended access and escalation of privilege.\nFor additional information, please review the authorization section of the https://developer.salesforce.com/docs/atlas.en-us.sfdx_dev.meta/sfdx_dev/sfdx_dev_auth_web_flow.htm."
  ]
}

ST

JSONDecodeError: Expecting value: line 1 column 1 (char 0)